In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR100
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from datetime import datetime

# Defining model
arch = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

class VGGNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(arch)
        self.fcs = nn.Sequential(
            nn.Linear(in_features=512*1*1, out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, arch):
        layers = []
        in_channels = self.in_channels

        for x in arch:
            if type(x) == int:
                out_channels = x
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

# Hyperparameters and settings

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
TRAIN_BATCH_SIZE = 64
VAL_BATCH_SIZE = 16
EPOCHS = 20

train_data = CIFAR100(root=".", train=True, transform=transforms.Compose([transforms.ToTensor()]), download=True)
val_data = CIFAR100(root=".", train=False, transform=transforms.Compose([transforms.ToTensor()]), download=True)

train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=VAL_BATCH_SIZE, shuffle=True, num_workers=2)

num_train_batches = len(train_loader)
num_val_batches = len(val_loader)

# Training and Val Loop

model = VGGNet(3, 100).to(device)  # Updated number of classes to 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def train_val():
    for epoch in range(1, EPOCHS + 1):
        print(f"Epoch: {epoch}/{EPOCHS}")
        model.train()
        total_loss = 0
        for data in tqdm(train_loader):
            image, target = data[0], data[1]
            image, target = image.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, target)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        print(f"Loss : {total_loss / num_train_batches}")

        save_path = os.path.join("trained_models", f'{datetime.now().strftime("%m%d_%H%M%S")}_{epoch}.pth')

        if not os.path.exists("trained_models"):
            os.makedirs("trained_models")
        if epoch % 5 == 0:
            torch.save(model.state_dict(), save_path)

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for data in val_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= num_val_batches
        print(f"Validation Loss: {val_loss}")

        train_accuracy = calculate_accuracy(train_loader, model)
        val_accuracy = calculate_accuracy(val_loader, model)
        print(f"Training Accuracy: {train_accuracy}%")
        print(f"Validation Accuracy: {val_accuracy}%")

        scheduler.step(val_loss)

train_val()


cuda


100%|██████████| 169001437/169001437 [00:04<00:00, 40025723.94it/s]


Extracting ./cifar-100-python.tar.gz to .
Files already downloaded and verified
Epoch: 1/20


100%|██████████| 782/782 [00:30<00:00, 25.27it/s]

Loss : 4.071591485796682


...

100%|██████████| 782/782 [00:29<00:00, 26.72it/s]

Loss : 0.42868658155202866


Validation Loss: 2.2181636924266814
Training Accuracy: 89.916%
Validation Accuracy: 52.85%
Epoch: 20/20


100%|██████████| 782/782 [00:29<00:00, 26.78it/s]


Loss : 0.40072877610773994
Validation Loss: 2.2080140793323517
Training Accuracy: 91.128%
Validation Accuracy: 53.13%


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
import torch.utils.data as data

# Load the pre-trained VGG-16 model
pretrained_model = models.vgg16(pretrained=True)

# Modify the classifier to accommodate 100 classes
pretrained_model.classifier[6] = nn.Linear(4096, 100)

# Transfer the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
pretrained_model.to(device)
# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Train the model
train_model(pretrained_model, train_loader, criterion, optimizer, num_epochs=50)
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    loss = running_loss / len(test_loader)
    return accuracy, loss

# Evaluate the model
train_accuracy, train_loss = evaluate_model(pretrained_model, train_loader, criterion)
test_accuracy, test_loss = evaluate_model(pretrained_model, test_loader, criterion)

print(f'Training Accuracy: {train_accuracy}%, Training Loss: {train_loss}')
print(f'Test Accuracy: {test_accuracy}%, Test Loss: {test_loss}')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50], Loss: 2.7244497824203022
Epoch [2/50], Loss: 1.938762121188366
Epoch [3/50], Loss: 1.6785478582772453
Epoch [4/50], Loss: 1.5024753397383044
Epoch [5/50], Loss: 1.361984170001486
Epoch [6/50], Loss: 1.2822484552402935
Epoch [7/50], Loss: 1.1966991198947057
Epoch [8/50], Loss: 1.1165104828527213
Epoch [9/50], Loss: 1.0606586956002217
Epoch [10/50], Loss: 1.0017253799206765
Epoch [11/50], Loss: 0.9530996014090145
Epoch [12/50], Loss: 0.9256395188438923
Epoch [13/50], Loss: 0.8755124939219726
Epoch [14/50], Loss: 0.8376456636297124
Epoch [15/50], Loss: 0.800411134348501
Epoch [16/50], Loss: 0.7691053159706428
Epoch [17/50], Loss: 0.7324947165253827
Epoch [18/50], Loss: 0.71348589048971
Epoch [19/50], Loss: 0.6970498329385773
Epoch [20/50], Loss: 0.6614624529390993
Epoch [21/50], Loss: 0.6482480296393489
Epoch [22/50], Loss: 0.6363333194609493
Epoch [23/50], Loss: 0.610279957732886
Epoch [24/50], Loss

In [ ]:
!pip install torch torchvision

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
import torch.utils.data as data

# Load the pre-trained VGG-16 model
pretrained_model = models.vgg16(pretrained=True)

# Freeze all the fully connected layers except the last one
for param in pretrained_model.classifier[:-1].parameters():
    param.requires_grad = False

# Modify the classifier to accommodate 100 classes
pretrained_model.classifier[6] = nn.Linear(4096, 100)

# Transfer the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
pretrained_model.to(device)

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Train the model
train_model(pretrained_model, train_loader, criterion, optimizer, num_epochs=50)

# Evaluation function
def evaluate_model(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    loss = running_loss / len(loader)
    return accuracy, loss

# Evaluate the model
train_accuracy, train_loss = evaluate_model(pretrained_model, train_loader, criterion)
test_accuracy, test_loss = evaluate_model(pretrained_model, test_loader, criterion)

print(f'Training Accuracy: {train_accuracy}%, Training Loss: {train_loss}')
print(f'Test Accuracy: {test_accuracy}%, Test Loss: {test_loss}')






Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50], Loss: 2.7554748372348676
Epoch [2/50], Loss: 1.9268384549928748
Epoch [3/50], Loss: 1.6630108530259193
Epoch [4/50], Loss: 1.5015466289447092
Epoch [5/50], Loss: 1.3782220270932484
Epoch [6/50], Loss: 1.2645685928861807
Epoch [7/50], Loss: 1.1803875240828374
Epoch [8/50], Loss: 1.1108896209455816
Epoch [9/50], Loss: 1.0531237052224787
Epoch [10/50], Loss: 0.9950851465734984
Epoch [11/50], Loss: 0.9370342691231262
Epoch [12/50], Loss: 0.9101732301590083
Epoch [13/50], Loss: 0.8646299855788345
Epoch [14/50], Loss: 0.8380627183962965
Epoch [15/50], Loss: 0.7891150593300305
Epoch [16/50], Loss: 0.7594345751625803
Epoch [17/50], Loss: 0.7251285621729653
Epoch [18/50], Loss: 0.7126562694454437
Epoch [19/50], Loss: 0.67493488675798
Epoch [20/50], Loss: 0.6498098653143324
Epoch [21/50], Loss: 0.632088006609846
Epoch [22/50], Loss: 0.6174919390312547
Epoch [23/50], Loss: 0.5925803928424025
Epoch [24/50], L